In [ ]:
# Author: Robert Guthrie
import sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

torch.manual_seed(1)

In [ ]:
def split_train_test_val(df):
    ind_year = np.where(np.array(traj.index.names)=='year')[0][0]
    train_df = df.loc[df.index.get_level_values(ind_year)<=2010]
    val_df = df.loc[(df.index.get_level_values(ind_year)>2010) & (df.index.get_level_values(ind_year)<=2014)]
    test_df = df.loc[df.index.get_level_values(ind_year)>2014]
    return train_df, val_df, test_df

# Load data

In [ ]:
inun_csv_list = glob.glob('../data/state_county_csvs/counties/inun_frac*')

In [ ]:
rand_csv = np.random.choice(inun_csv_list)

In [ ]:
joined_df = read_join_csv(rand_csv)

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    old_cols = data.columns
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (old_cols[j], i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('%s(t)' % (old_cols[j])) for j in range(n_vars)]
        else:
            names += [('%s(t+%d)' % (old_cols[j], i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
traj = joined_df.loc[joined_df.index.get_level_values(0)[0]]
traj = traj[['inundation', 'acres', 'vpd', 'temp','precip']]
n_features = traj.shape[1]
traj['inundation'].plot()

In [ ]:
traj = series_to_supervised(traj)
traj = traj.drop(columns=traj.columns[range(-(n_features-1),0)])

# Set params, start training

In [ ]:
train, val, test = split_train_test_val(traj)
train = scaler.fit_transform(train)
val = scaler.transform(val)
test = scaler.transform(test)
train_X, train_y = train[:, :-1], train[:, -1]
val_X, val_y = val[:, :-1], val[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

In [ ]:
batch_size = 8
batch_size_val = 8
lstm_input_size = traj.shape[1]-1
hidden_dim = 50
output_dim=1
num_layers=1
learning_rate = 0.01
num_epochs =200

In [ ]:
def tensorfy(x, y, batch_size):
    batch_starts = np.arange(0, x.shape[0], batch_size)
    x_tensor = [torch.tensor(np.array(x[i:(i+batch_size)])).float() for i in batch_starts]
    if len(x_tensor[-1]) < batch_size: # drop last batch if not even
        y = y[:-len(x_tensor[-1])]
        x_tensor = x_tensor[:-1]
    return x_tensor, y

In [ ]:
train_X_tensor, train_y = tensorfy(train_X, train_y, batch_size)
val_X_tensor, val_y = tensorfy(val_X, val_y, batch_size_val)
test_X_tensor, test_y = tensorfy(test_X, test_y, batch_size_val)


In [ ]:
# Here we define our model as a class
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(torch.cat(input).view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred.view(-1)


model = LSTM(input_dim = lstm_input_size,
             hidden_dim=50,
             batch_size=batch_size,
             output_dim=1,
             num_layers=1)

In [ ]:
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
#####################
# Train model
#####################

hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    model.batch_size=batch_size
    train_pred = model(train_X_tensor)

    loss = loss_fn(train_pred, torch.tensor(train_y).float())
    
    model.batch_size=batch_size_val
    val_pred = model(val_X_tensor)
    val_loss = loss_fn(val_pred, torch.tensor(val_y).float())
    print("Epoch ", t, "Train MAE: ", loss.item(), ", Val MAE: ", val_loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

In [ ]:
plt.scatter(train_pred.detach().numpy(), train_y)

In [ ]:
pd.DataFrame({'Pred':train_pred.detach().numpy(), 'True':train_y}).plot(xlim=(200,300))

In [ ]:
plt.scatter(model(val_X_tensor).detach().numpy(), val_y)
pd.DataFrame({'Pred':model(val_X_tensor).detach().numpy(), 'True':val_y}).plot()

In [ ]:
plt.scatter(model(test_X_tensor).detach().numpy(), test_y)
pd.DataFrame({'Pred':model(test_X_tensor).detach().numpy(), 'True':test_y}).plot()